In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Chatbot with no memory

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

In [3]:
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

### Chatbot with memory

In [5]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="""list the names of people who have said hello to you in our conversation so far.\n
List the names following the below format:\n

[<Name 1>, <Name 2>, <Name 3>, ...]

For example, if the names of people who have said hello to you in our conversation so far are Alice and Bob, you should list them as follows:
    
[Alice, Bob]                  
""")],
    config=config,
)

response.content

In [ ]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

### Chatbot with memory and system prompt

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an unhelpful assisstant. You should make up incorrect answers to questions.\n
              Do not use negatives in your answer, but instead create a made up answer.\n
              Make your answers sound as real as possible, whilst being completely incorrect.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="what is the purpose of a knife and fork?")],
    config=config,
)

response.content

### Chatbot with memory, system prompt and inputs to prompt template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an helpful assisstant. You should provide helpful answers to questions.\n
              Use the tone of {tone} in your answers.\n""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "tone": "pirate"},
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hole mi amigo, cual es mi nombre?")], "tone": "pirate"},
    config=config,
)

response.content

## Trimmer to limit number of tokens in the memory

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "tone": "alien",
    },
    config=config,
)

response.content


### Streaming 

In [ ]:
config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm todd.")],
        "tone": "dungeon final enemy boss at start of epic battle",
    },
    config=config,
):
    print(r.content, end="")